In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pandas import DataFrame

사용자가 별점 준 내용으로 데이터를 받아들일것이다

In [14]:
movies=pd.read_csv("movies.csv")
ratings=pd.read_csv("ratings.csv")

# 1. Pandas 를 통해 포레스트 검프와 비슷한 영화 찾기

In [15]:
movie_data=pd.merge(ratings,movies, on="movieId") # movieid를 기준으로 데이터 파일 합치기
movie_data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


#제목별로 점수가 높은거 평균의 순차적으로 나타내기

In [16]:
movie_data_group=movie_data.groupby("title")["rating"].mean().sort_values(ascending=False)
movie_data_group.head(10)

title
Karlson Returns (1970)                           5.0
Winter in Prostokvashino (1984)                  5.0
My Love (2006)                                   5.0
Sorority House Massacre II (1990)                5.0
Winnie the Pooh and the Day of Concern (1972)    5.0
Sorority House Massacre (1986)                   5.0
Bill Hicks: Revelations (1993)                   5.0
My Man Godfrey (1957)                            5.0
Hellbenders (2012)                               5.0
In the blue sea, in the white foam. (1984)       5.0
Name: rating, dtype: float64

In [17]:
movie_data.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres'], dtype='object')

In [18]:
movie_rating=movie_data.pivot_table(index="userId",columns="title",values="rating")
movie_rating.columns

Index([''71 (2014)', ''Hellboy': The Seeds of Creation (2004)',
       ''Round Midnight (1986)', ''Salem's Lot (2004)',
       ''Til There Was You (1997)', ''Tis the Season for Love (2015)',
       ''burbs, The (1989)', ''night Mother (1986)',
       '(500) Days of Summer (2009)', '*batteries not included (1987)',
       ...
       'Zulu (2013)', '[REC] (2007)', '[REC]² (2009)',
       '[REC]³ 3 Génesis (2012)',
       'anohana: The Flower We Saw That Day - The Movie (2013)',
       'eXistenZ (1999)', 'xXx (2002)', 'xXx: State of the Union (2005)',
       '¡Three Amigos! (1986)', 'À nous la liberté (Freedom for Us) (1931)'],
      dtype='object', name='title', length=9719)

In [19]:
forrest_gump_rating=movie_rating["Forrest Gump (1994)"]
movies_like_forrest=movie_rating.corrwith(forrest_gump_rating)

corr_forrest=pd.DataFrame(movies_like_forrest,columns=["Correlation"])
print(corr_forrest.sort_values("Correlation",ascending=False).head(20))

C:\Users\Affinity\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\Affinity\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


                                                    Correlation
title                                                          
Lost & Found (1999)                                         1.0
Century of the Self, The (2002)                             1.0
The 5th Wave (2016)                                         1.0
Play Time (a.k.a. Playtime) (1967)                          1.0
Memories (Memorîzu) (1995)                                  1.0
Playing God (1997)                                          1.0
Killers (2010)                                              1.0
Girl Walks Home Alone at Night, A (2014)                    1.0
Tampopo (1985)                                              1.0
Cercle Rouge, Le (Red Circle, The) (1970)                   1.0
The Boss (2016)                                             1.0
Poison Ivy: New Seduction (1997)                            1.0
Tale of Two Sisters, A (Janghwa, Hongryeon) (2003)          1.0
Welcome to the Jungle (2013)            

In [20]:
movies_like_forrest

title
'71 (2014)                                                                                       NaN
'Hellboy': The Seeds of Creation (2004)                                                          NaN
'Round Midnight (1986)                                                                           NaN
'Salem's Lot (2004)                                                                              NaN
'Til There Was You (1997)                                                                        NaN
'Tis the Season for Love (2015)                                                                  NaN
'burbs, The (1989)                                                                          0.197712
'night Mother (1986)                                                                             NaN
(500) Days of Summer (2009)                                                                 0.234095
*batteries not included (1987)                                                       

# 2. CF-based 추천 시스템 구현

In [21]:
user=pd.read_csv("u.user", sep="|",encoding="latin-1",names=["user_id","age","sex","occupation","zip_code"]) # sep을 찾아서 읽기
data=pd.read_csv("u.data",sep="\t",encoding="latin-1",names=["user_id","movie_id","rating","unix_time"])
item=pd.read_csv("u.item",sep="|",encoding="latin-1",header=None) # 전체를 읽어들이는것

In [22]:
user

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201
8,9,29,M,student,01002
9,10,53,M,lawyer,90703


In [23]:
item.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [24]:
data.head()

,user_id,movie_id,rating,unix_time
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [25]:
# user의 userID 와 movieID count

In [26]:
data.columns

Index(['user_id', 'movie_id', 'rating', 'unix_time'], dtype='object')

In [27]:
n_user_id=data.user_id.nunique()
n_user_id

943

In [28]:
n_movie_id=data.movie_id.nunique()
n_movie_id

1682

#data matrics 만들기, 행, 렬 에 기반하여 값을 넣어주기.

In [29]:
data_matrix=np.zeros((n_user_id,n_movie_id)) ##0으로 초기화

for line in data.itertuples():
    data_matrix[line[1]-1, line[2]-1]=line[3]
    
    ## data.itertuples는 한 줄의 모든값을 불러온다. 이 값들을 data_matrix에 넣어준다.

In [30]:
data

,user_id,movie_id,rating,unix_time
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [31]:
data_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

# 사용자가 얼마나 영화m을 좋아하는지 예측하는 함수

In [32]:
import sklearn

In [33]:
# similarity는 행끼리 비교해서. 
# item-similarity 는 user_similiarity=pairwise_distances(data_matrix.T,metric="cosine") # transpos

In [34]:
from sklearn.metrics.pairwise import pairwise_distances
user_similiarity=pairwise_distances(data_matrix,metric="cosine")# cosine 메트릭을 사용하여 daata_metrics의 거리를 계신
user_similiarity.shape
user_similiarity

# 행,렬 같은 것들은 0 으로 나옴, 같은 자리니까.

array([[0.        , 0.83306902, 0.95254046, ..., 0.85138306, 0.82049212,
        0.60182526],
       [0.83306902, 0.        , 0.88940868, ..., 0.83851522, 0.82773219,
        0.89420212],
       [0.95254046, 0.88940868, 0.        , ..., 0.89875744, 0.86658385,
        0.97344413],
       ...,
       [0.85138306, 0.83851522, 0.89875744, ..., 0.        , 0.8983582 ,
        0.90488042],
       [0.82049212, 0.82773219, 0.86658385, ..., 0.8983582 , 0.        ,
        0.81753534],
       [0.60182526, 0.89420212, 0.97344413, ..., 0.90488042, 0.81753534,
        0.        ]])

In [35]:
# 행렬전체를 받아서 행렬전체를 주는것

In [36]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        
    return pred

In [37]:
data_matrix.mean(axis=1)## 1차원을 2차원으로 바꿔주라.

array([0.58382878, 0.13674197, 0.08977408, 0.06183115, 0.29904875,
       0.45600476, 0.95005945, 0.13317479, 0.05588585, 0.46016647,
       0.37277051, 0.13317479, 1.17122473, 0.23840666, 0.17776457,
       0.36028537, 0.05053508, 0.6391201 , 0.04221165, 0.08858502,
       0.28418549, 0.25505351, 0.32639715, 0.17479191, 0.18787158,
       0.18727705, 0.04815696, 0.17479191, 0.07372176, 0.09631391,
       0.08382878, 0.08085612, 0.0529132 , 0.04815696, 0.04458977,
       0.0451843 , 0.12366231, 0.26753864, 0.04934602, 0.06004756,
       0.11652794, 0.40546968, 0.48751486, 0.32758621, 0.10285375,
       0.06599287, 0.05410226, 0.14863258, 0.343044  , 0.05053508,
       0.04875149, 0.14209275, 0.06361474, 0.14268728, 0.04102259,
       0.40487515, 0.22057075, 0.35136742, 0.89357907, 0.50535077,
       0.03686088, 0.45541023, 0.17241379, 0.42627824, 0.18727705,
       0.078478  , 0.06718193, 0.06123662, 0.14863258, 0.26991677,
       0.08561237, 0.31093936, 0.14625446, 0.08442331, 0.15398

In [38]:
data_matrix.mean(axis=1)[:,np.newaxis]## 1차원을 2차원으로 바꿔주라.

array([[0.58382878],
       [0.13674197],
       [0.08977408],
       [0.06183115],
       [0.29904875],
       [0.45600476],
       [0.95005945],
       [0.13317479],
       [0.05588585],
       [0.46016647],
       [0.37277051],
       [0.13317479],
       [1.17122473],
       [0.23840666],
       [0.17776457],
       [0.36028537],
       [0.05053508],
       [0.6391201 ],
       [0.04221165],
       [0.08858502],
       [0.28418549],
       [0.25505351],
       [0.32639715],
       [0.17479191],
       [0.18787158],
       [0.18727705],
       [0.04815696],
       [0.17479191],
       [0.07372176],
       [0.09631391],
       [0.08382878],
       [0.08085612],
       [0.0529132 ],
       [0.04815696],
       [0.04458977],
       [0.0451843 ],
       [0.12366231],
       [0.26753864],
       [0.04934602],
       [0.06004756],
       [0.11652794],
       [0.40546968],
       [0.48751486],
       [0.32758621],
       [0.10285375],
       [0.06599287],
       [0.05410226],
       [0.148

# 성능평가 MAE

In [39]:
user_prediction=predict(data_matrix,user_similiarity,type="user")
#통째로 다 계산 해주는 프레딕트 함수

In [40]:
from sklearn.metrics import mean_absolute_error
print("MAE:  ", mean_absolute_error(data_matrix,user_prediction))

MAE:   0.41217773816161574


In [41]:
# 실제와 예상값 비교

In [42]:
print("Actual rating:", data_matrix[4,2])
print("Predict rating:", user_prediction[4,2])

Actual rating: 0.0
Predict rating: 0.35442233234815124


# 3.Content Based Recommender

#imdb의 top rated 영화들 사용
#영화추천문제를 CB로 구성하기

#배우, 감독이름을 가공, 줄거리 키워드 추출
#영화별로 키워드별로 카운트 행렬 생성
#코사인 유사도를 이용 
#특정영화와 유사한 영화 추천

#개인의 신원 정보는 사용하지 않으니까, 추천? 이랄 수는 없지만 영화들의 벡터를 비교해서 가장 유사한것 찾아보는것

# 사실 고도의 Pandas 의 활용.

In [82]:
!pip install rake_nltk

In [93]:
import numpy as np
import pandas as pd

from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [131]:
data=pd.read_csv("IMDBTOP250.csv")

In [128]:
data.head(5)
data.columns

Index(['Unnamed: 0', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre',
       'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards',
       'Poster', 'Ratings.Source', 'Ratings.Value', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'tomatoMeter', 'tomatoImage',
       'tomatoRating', 'tomatoReviews', 'tomatoFresh', 'tomatoRotten',
       'tomatoConsensus', 'tomatoUserMeter', 'tomatoUserRating',
       'tomatoUserReviews', 'tomatoURL', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response'],
      dtype='object')

# Check
#각각의 열에 있는 값들을 Actors는 , 를 기준으로 나누어서 3명까지 불러오고, 
#director는 스페이스를 기준으로 나눠서 불러온다.

In [133]:
data["Actors"]=data["Actors"].map(lambda x:x.lower())
data["Director"]=data["Director"].map( lambda x:x.lower())

In [135]:
data["Actors"]=data["Actors"].map(lambda x:x.split(",")[:3])
data["Director"]=data["Director"].map(lambda x: x.split(" "))

In [134]:
data.iterrows

<bound method DataFrame.iterrows of      Unnamed: 0                                              Title  Year  \
0             1                           The Shawshank Redemption  1994   
1             2                                      The Godfather  1972   
2             3                             The Godfather: Part II  1974   
3             4                                    The Dark Knight  2008   
4             5                                       12 Angry Men  1957   
5             6                                   Schindler's List  1993   
6             7      The Lord of the Rings: The Return of the King  2003   
7             8                                       Pulp Fiction  1994   
8             9                                         Fight Club  1999   
9            10  The Lord of the Rings: The Fellowship of the Ring  2001   
10           11                                       Forrest Gump  1994   
11           12     Star Wars: Episode V - The Empir

# Check
#data.itterrow 해서 각 인덱스별 값들을 나누고, r각각의 행에 있는 Actors 와 DIrector의 값을 가공. 소문자로 변환뒤 한단어로 쭉 붙임

#Genre 는 소문자로 변환뒤에 리스트로 만들어줌. , 를 기준으로 띄어서

In [172]:
# for idx,row in data.iterrows():
#     t=[]
#     row["Actors"]=[t.join(x) for x in row["Actors"]]
#     row["Director"]="".join(row["Director"])

In [170]:
for i in range(len(data['Actors'])):
    actors_len = len(data['Actors'][i])
    for j in range(actors_len):
        data['Actors'][i][j] = data['Actors'][i][j].replace(' ', '')

In [ ]:
for i in range(len(data['Director'])):
    t=[]
    data['Director'][i] = t.append(data['Director'][i])

C:\Users\Affinity\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data["Actors","Director","Genre"]

In [137]:
data["Genre"]=data["Genre"].map(lambda x:x.lower().split(","))

# Check
#줄거리 plot에서 키워드 추출

In [139]:
data["Key_words"]=""

for idx, row in data.iterrows():
    plot=row["Plot"]
    r=Rake()
    r.extract_keywords_from_text(plot)
    key_words_dict_scores=r.get_word_degrees()
    row["Key_words"]=list(key_words_dict_scores.keys())


In [140]:
data['Bag of words']=""
columns=data.columns

for idx,row in data.iterrows():
    words=""
    
    for col in columns:
        if col!="Director":
            words=words + " ".join(row[col])+ " "
        else:
            words=words+row[col]+ " "
    row["Bag of words"]=words

TypeError: can only join an iterable

In [142]:
data.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response,Key_words,Bag of words
0,1,The Shawshank Redemption,1994,R,14-Oct-94,142 min,"[crime, drama]","[frank, darabont]","Stephen King (short story ""Rita Hayworth and S...","[tim robbins, morgan freeman, bob gunton]",...,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27-Jan-98,NaN,Columbia Pictures,NaN,True,,
1,2,The Godfather,1972,R,24-Mar-72,175 min,"[crime, drama]","[francis, ford, coppola]","Mario Puzo (screenplay), Francis Ford Coppola ...","[marlon brando, al pacino, james caan]",...,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09-Oct-01,NaN,Paramount Pictures,http://www.thegodfather.com,True,,
2,3,The Godfather: Part II,1974,R,20-Dec-74,202 min,"[crime, drama]","[francis, ford, coppola]","Francis Ford Coppola (screenplay), Mario Puzo ...","[al pacino, robert duvall, diane keaton]",...,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24-May-05,NaN,Paramount Pictures,http://www.thegodfather.com/,True,,
3,4,The Dark Knight,2008,PG-13,18-Jul-08,152 min,"[action, crime, drama]","[christopher, nolan]","Jonathan Nolan (screenplay), Christopher Nolan...","[christian bale, heath ledger, aaron eckhart]",...,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09-Dec-08,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True,,
4,5,12 Angry Men,1957,APPROVED,01-Apr-57,96 min,"[crime, drama]","[sidney, lumet]","Reginald Rose (story), Reginald Rose (screenplay)","[martin balsam, john fiedler, lee j. cobb]",...,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06-Mar-01,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True,,
